In [96]:
import numpy as np
import CoolProp.CoolProp as CP
import math
import plotly.offline as pyo
import plotly.graph_objects as go

#GEOMETRY FROM NTOPOLOGY
vthex = 195461.4765 #mm³ total volume of the hex core
vtf2 = 125578.5438 #mm³ total volume of the mid core
vtf3 = 83879.8214 #mm³ total volume of the outer core
vf1 = 44195.7598 #mm³ total volume of the fluid1
vf2 = 27184.1405 #mm³ total volume of the fluid2
vf3 = 14342.9852 #mm³ total volume of the fluid3
Lf1f2 = 100/1000 #m
Lf3 = 80/1000 #m

f1sa = 94009.046 #mm² f1 wetted area
f2sa = 50021.1927 #mm² f2 wetted area
f3sa = 36110.0301 #mm² f3 wetted area

#porosity
epsif1 = vf1/vthex
epsif2 = vf2/vtf2
epsif3 = vf3/vtf3

print(epsif1)

0.22610982271997726


In [97]:

#air props
muair = CP.PropsSI('V', 'T', 300, 'P', 101325, 'Air')  # air dynamic viscosity
rhoair = CP.PropsSI('D', 'T', 300, 'P', 101325, 'Air')  # air density

# water props
muw = CP.PropsSI('V', 'T', 300, 'P', 101325, 'Water')  # water dyn visc
rhow = CP.PropsSI('D', 'T', 300, 'P', 101325, 'Water')  # water density

mair = np.linspace(0.05, 0.1, 10) #kg/s
mw1 = np.linspace(0.01, 0.1, 10) #kg/s
mw2 = np.linspace(0.01, 0.1, 10) #kg/s

Acs_fluid1 = vf1 / Lf1f2
Acs_fluid2 = vf2 / Lf1f2
Acs_fluid3 = vf3 / Lf3

Var = mair / (Acs_fluid1*rhoair)
Vw1 = mw1 / (Acs_fluid2*rhow)
Vw2 = mw2 / (Acs_fluid3*rhow)

def Dh(vf, afs): #OBTIDO NO ARTIGO QUE PROF KLEBER ENVIOU (NÃO AUTORIZADO COMPARTILHAMENTO)
    '''
    vf = representa o volume de fluído
    afs = área de interface solido/fluido ( verificar a definição no artigo nao publicado, ambas equações são iguais)
    '''
    Dh = (4*vf)/afs
    return Dh
#hydraulic diameter Dh
Dhf1 = Dh(vf1, f1sa)
Dhf2 = Dh(vf2, f2sa)
Dhf3 = Dh(vf3, f3sa)

def Re(m,Dh,mu,Across): #reynolds
    Re = (m*Dh)/(Across*mu)
    return Re
#reynolds calculation Re(m,Dh,mu,Across)
Ref1 = Re(mair, Dhf1, Acs_fluid1,muair)
Ref2 = Re(mw1, Dhf2, Acs_fluid2, muw)
Ref3 = Re(mw2, Dhf3, Acs_fluid3, muw)

print(Acs_fluid1)

441957.598


In [98]:

#------------- f CORRELATIONS --------------------#
#F CORRELATION_1
def fcorrelation1(Re): # kaixin Yan et al. 2024 correlation
    f = 2.577*(np.power(Re, -0.095)) 
    return f
f1air = fcorrelation1(Ref1)
f1w1 = fcorrelation1(Ref2)
f1w2 = fcorrelation1(Ref3)

#F CORRELATION_2
def fcorrelation2(Re): #kaixin Yan et al #GYROID AS PRINTED
    f = 2.756*(np.power(Re, -0.166))  
    return f
f2air = fcorrelation2(Ref1)
f2w1 = fcorrelation2(Ref2)
f2w2 = fcorrelation2(Ref3)

#F CORRELATION_3
def fcorrelation3(Re): # kaixin Yan et al #GYROID AS POLISHED 
    f = 2.344*(np.power(Re, -0.081)) 
    return f
f3air = fcorrelation3(Ref1)
f3w1 = fcorrelation3(Ref2)
f3w2 = fcorrelation3(Ref3)



#--------------------PRESSURE DROP------------------------#
def dP(f,L,epsi,Dh,rho,V): #REYNOLDS et al.
    dP = (f*L*2*rho*(np.power(V, 2))) / (np.power(epsi, 2)*Dh) 
    return dP

# pressuredrop for fcorrelation1
Dp_fcor1_f1 = dP(Lf1f2, epsif1, Dhf1, rhoair, Var, f1air)
Dp_fcor1_f2 = dP(Lf1f2, epsif2, Dhf2, rhow, Vw1, f1w1)
Dp_fcor1_f3 = dP(Lf3, epsif3, Dhf3, rhow, Vw2, f1w2)

# pressuredrop for fcorrelation2
Dp_fcor2_f1 = dP(Lf1f2, epsif1, Dhf1, rhoair, Var, f2air)
Dp_fcor2_f2 = dP(Lf1f2, epsif2, Dhf2, rhow, Vw1, f2w1)
Dp_fcor2_f3 = dP(Lf3, epsif3, Dhf3, rhow, Vw2, f2w2)

# pressuredrop for fcorrelation3
Dp_fcor3_f1 = dP(Lf1f2, epsif1, Dhf1, rhoair, Var, f3air)
Dp_fcor3_f2 = dP(Lf1f2, epsif2, Dhf2, rhow, Vw1, f3w1)
Dp_fcor3_f3 = dP(Lf3, epsif3, Dhf3, rhow, Vw2, f3w2)

In [99]:
# PLOT dP/Re for fcorrelation1
trace1 = go.Scatter(x=Ref1, y=Dp_fcor1_f1, mode='markers', name='dP_Air_fcorrelation1', marker=dict(color='blue'))
trace2 = go.Scatter(x=Ref1, y=Dp_fcor2_f1, mode='markers', name='dP_Air_fcorrelation2', marker=dict(color='red'))
trace3 = go.Scatter(x=Ref1, y=Dp_fcor3_f1, mode='markers', name='dP_Air_fcorrelation3', marker=dict(color='green'))

layout = go.Layout(title='Dp/Re', xaxis=dict(title='Reynolds Number'), yaxis=dict(title='Pressure Drop (dP)'))

fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
pyo.plot(fig)

print(f2air)

[5.78562483 5.6853149  5.59607258 5.51582448 5.44301998 5.37647054
 5.31524608 5.2586057  5.20595013 5.15678813]


In [100]:

# PLOT dP/Re for fcorrelation1
trace1 = go.Scatter(x=Ref2, y=Dp_fcor1_f2, mode='markers', name='dP_WaterF2_fcorrelation1', marker=dict(color='blue'))
trace2 = go.Scatter(x=Ref2, y=Dp_fcor2_f2, mode='markers', name='dP_WaterF2_fcorrelation2', marker=dict(color='red'))
trace3 = go.Scatter(x=Ref2, y=Dp_fcor3_f2, mode='markers', name='dP_WaterF2_fcorrelation3', marker=dict(color='green'))

layout = go.Layout(title='Dp/Re', xaxis=dict(title='Reynolds Number'), yaxis=dict(title='Pressure Drop (dP)'))

fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
pyo.plot(fig)

print(f1air)

[3.93941914 3.90018519 3.86503049 3.83321352 3.80417608 3.77748776
 3.75280985 3.72987117 3.70845131 3.68836881]


In [101]:
# PLOT dP/Re for fcorrelation1
trace1 = go.Scatter(x=Ref3, y=Dp_fcor1_f3, mode='markers', name='dP_WaterF3_fcorrelation1', marker=dict(color='blue'))
trace2 = go.Scatter(x=Ref3, y=Dp_fcor2_f3, mode='markers', name='dP_WaterF3_fcorrelation2', marker=dict(color='red'))
trace3 = go.Scatter(x=Ref3, y=Dp_fcor3_f3, mode='markers', name='dP_WaterF3_fcorrelation3', marker=dict(color='green'))

layout = go.Layout(title='Dp/Re', xaxis=dict(title='Reynolds Number'), yaxis=dict(title='Pressure Drop (dP)'))

fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
pyo.plot(fig)

print(f1air)

[3.93941914 3.90018519 3.86503049 3.83321352 3.80417608 3.77748776
 3.75280985 3.72987117 3.70845131 3.68836881]
